# 1. Describe the environment in the Nim learning model.

The environment of the learning model is the 'board' which consists of a list of stacks that contain some quantity of objects (3 stacks of at most 10 items in this case).

# 2. Describe the agent in the Nim learning model.

Our agent is a Q-Learning agent. This agent uses a table (*qtable*) of expected payoffs to make "educated" guesses about which stack to pull objects out of based on the highest expected reward based on the current state of the environment. The number of items pulled (*move*) is equal to a modulo of the index of the highest reward (*a*) by the maximum number of states of a stack, The stack pulled from (*pile*) is determined by an integer division of the index of the highest reward (*a*) by the maximum number of items in a stack.

If this procedure produces an illegal move then the agent will instead make a random legal move. 

# 3. Describe the reward and penalty in the Nim learning model.

In this model, reward/penalty is determined by the function $\alpha * (reward + \gamma * q'_{best})$. In this equation, $\alpha$ adjust the gain for the learning function, $reward$ is gives a large boost to moves that result in a win, and the $\gamma *q'_{best}$ term assures that all legal moves get at least some reward (but less than a win). The $q'_{best}$ term also means that the model won't be able to learn moves until after a game has been won, as that is the only way to set any of the terms in *qtable* above zero (*qtable* is zeroed out at the beginning of the learning function).

Actions of the default qlearner:

* Win: $reward = 100$; $q'_{best}$ set to 0. Sets a move as the correct one for a win.
* Legal Move: $reward = 0$. Makes this move a fraction ($\gamma$) of the best move for that state.
* Loss: No special action.

# 4. How many possible states there could be in the Nim game with a maximum of 10 items per pile and 3 piles total?

The number of states is solvable as the product of the possible states of the stacks. The possible states of the stacks is just equal to the max occupancy plus one (for the empty state).

So in this case the possible number of states for 3 stacks ($st_1$, $st_2$, $st_3$) of at max $10$ items each is equal to $(st_1 + 1)(st_2 + 1)(st_2 + 1) = 11^3 = 1331$

# 5. How many possible actions there could be in the Nim game with 10 items per pile and 3 piles total?

The only action an agent may take is to take objects from a stack. The number of possible actions for each stack is then just the number of items in that stack. Therefore the total number of actions an agent may take is equal to the sum of the stack occupancies. In this case with 3 stacks of 10 items, we have $st_1 + st_2 + st_3 = 3 * 10 = 30$ possible actions.

# 6. Find a way to improve the provided Nim game learning model

Code taken/adapted from module 9 notebook.

In [1]:
import numpy as np
from functools import partial
from itertools import product
from os import mkdir, path
from random import randint, choice

# The number of piles is 3

# max number of items per pile
ITEMS_MX = 10

# accomadate multiple logs
log_path = './logs/'
if not path.isdir(log_path):
    mkdir(log_path)

# Initialize starting position
def init_game():
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(st):
    xored = st[0] ^ st[1] ^ st[2]
    if xored == 0:
        return nim_random(st)
    #
    for pile in range(3):
        s = st[pile] ^ xored
        if s <= st[pile]:
            return st[pile]-s, pile

# Random Nim player
def nim_random(_st):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

In [2]:
Engines = {'Random':nim_random, 'Guru':nim_guru}

def game(a, b):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[a] if side == 'A' else Engines[b]
        move, pile = engine(state)
        # print(state, move, pile)  # debug purposes
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side
        #
        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n, a, b):
    from collections import defaultdict
    wins = defaultdict(int)
    for i in range(_n):
        wins[game(a, b)] += 1
    # info
    print(f"{_n} games, {a:>13s} vs. {b:>13s}, {a} Winrate: {100 * wins['A'] / (wins['A'] + wins['B']):.2f}%")
    #
    return wins['A'], wins['B']

In [3]:
# Function to print the entire set of states
def qtable_log(_fn, qtable):
    fn = log_path + _fn
    
    with open(fn, 'w') as fout:
        s = 'state'
        for a in range(ITEMS_MX*3):
            move, pile = a%ITEMS_MX+1, a//ITEMS_MX
            s += ',%02d_%01d' % (move,pile)
        #
        print(s, file=fout)
        for i, j, k in [(i,j,k) for i in range(ITEMS_MX+1) for j in range(ITEMS_MX+1) for k in range(ITEMS_MX+1)]:
            s = '%02d_%02d_%02d' % (i,j,k)
            for a in range(ITEMS_MX*3):
                r = qtable[i, j, k, a]
                s += ',%.1f' % r
            #
            print(s, file=fout)

### Default QLearner

Modified from module 9 notebook for modularity.

In [4]:
Alpha, Gamma, Reward = 1.0, 0.8, 100.0

def nim_qlearner(_st, qtable, explore):
   
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = (a%ITEMS_MX)+1, a//ITEMS_MX
    
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = explore(_st)  # exploration
    #
    return move, pile  # action

# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n, qtable, explore):
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=float)
    # play _n games
    for i in range(_n):
        # first state is starting position
        st1 = init_game()
        while True:  # while game not finished
            # make a random move - exploration
            move, pile = explore(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            if st2 == [0, 0, 0]:  # game ends
                qtable = qtable_update(qtable, Reward, st1, move, pile, 0)  # I won
                break  # new game
            #
            qtable = qtable_update(qtable, 0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            st1 = st2
            
    return qtable

# Equation 3 - update the qtable
def qtable_update(qtable, r, _st1, move, pile, q_future_best):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)
    return qtable

In [5]:
%%time
# Train and store
default_qtable = None
default_qtable = nim_qlearn(10000, qtable=default_qtable, explore=nim_random)
default_qlearner = partial(nim_qlearner, qtable=default_qtable, explore=nim_random)

# add to the Engines dictionary
Engines['DQlearner'] = default_qlearner

qtable_log('default_qtable.csv', default_qtable)

Wall time: 738 ms


## My Agents

Code is adapted from the module 9 notebook.

### Guru

Uses the *nim_guru()* function for exploration instead of just random (there will still be some randomness initially). THis makes a little more sense than leaving things completely up to chance as we know this game is mathematically solved.

This performs better than the default Qlearner, but if *nim_guru()* is allowed to go first this agent will not win the majority (vice-versa if this agent is allowed the first move versus *nim_guru()* it *will* win the majority).

In [6]:
%%time
# Train and store
guru_qtable = None
guru_qtable = nim_qlearn(10000, qtable=guru_qtable, explore=nim_guru)
guru_qlearner = partial(nim_qlearner, qtable=guru_qtable, explore=nim_guru)

# add to the Engines dictionary
Engines['guru_Qlearner'] = guru_qlearner

qtable_log('default_qtable.csv', guru_qtable)

Wall time: 686 ms


## FIGHT!

Pit agents against one another.

In [7]:
# LEEET'S GET READY TO RUUUUUMBLLLLLE!
prev_a = None
for a, b in product(Engines, repeat=2):
    if not prev_a:
        prev_a = a
    elif prev_a != a:
        # breaks our fights up by who goes first
        prev_a = a
        print('\n')
    
    play_games(1000, a, b)

1000 games,        Random vs.        Random, Random Winrate: 50.40%
1000 games,        Random vs.          Guru, Random Winrate: 0.60%
1000 games,        Random vs.     DQlearner, Random Winrate: 30.70%
1000 games,        Random vs. guru_Qlearner, Random Winrate: 1.30%


1000 games,          Guru vs.        Random, Guru Winrate: 99.90%
1000 games,          Guru vs.          Guru, Guru Winrate: 93.50%
1000 games,          Guru vs.     DQlearner, Guru Winrate: 99.90%
1000 games,          Guru vs. guru_Qlearner, Guru Winrate: 94.40%


1000 games,     DQlearner vs.        Random, DQlearner Winrate: 71.40%
1000 games,     DQlearner vs.          Guru, DQlearner Winrate: 2.60%
1000 games,     DQlearner vs.     DQlearner, DQlearner Winrate: 90.90%
1000 games,     DQlearner vs. guru_Qlearner, DQlearner Winrate: 9.10%


1000 games, guru_Qlearner vs.        Random, guru_Qlearner Winrate: 99.80%
1000 games, guru_Qlearner vs.          Guru, guru_Qlearner Winrate: 93.90%
1000 games, guru_Qlearner vs